# 網頁url寫入文件檔 一個月一個檔

In [1]:
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
#製造日期的迴圈
startdate = dt.datetime(2016, 3,1)  #輸入開始日期
endate = dt.datetime(2016, 3,31)       #輸入結束日期
totaldays = (endate - startdate).days + 1  #計算有幾天
start = (startdate ).date().strftime("%Y%m")  

with open('C:/Users/BIG DATA/Desktop/python/essay/{}.txt'.format(start), 'w') as f:
    for daynumber in range(totaldays):
        datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y%m%d")  
    
        url = 'http://news.ltn.com.tw/newspaper/focus/{}'.format(datestring)
        res = requests.get(url)
        soup = bs(res.text)

        Times = 'http://news.ltn.com.tw'
        sort = soup.select('.newspaper a')
        
        for a in xrange(0, len(sort)):
            url2 = Times + sort[a]['href'] #分類網址
            res2 = requests.get(url2)
            soup2 = bs(res2.text)
            #print len(soup2.select('.p_num'))
            if len(soup2.select('.p_num')) == 1 : #判斷此類別新聞是否一頁
                title = soup2.select('.picword')
                title_num = len(soup2.select('.picword')) #計算有幾篇文章(每天新聞數量不同)
                for i in xrange(0 , title_num):   
                    url1=Times+title[i]['href']   #文章網址
                    f.write(url1  + '\n')  #寫入text
            else:
                for page1 in xrange (1,len(soup2.select('.p_num'))+1): #此類別新聞超過一頁時執行
                    url3= url2+'?page={}'.format(page1)
                    #print url3
                    res3 = requests.get(url3)
                    soup3 = bs(res3.text)
                    title = soup3.select('.picword')
                    title_num1 = len(soup3.select('.picword')) #計算有幾篇文章(每天新聞數量不同)
                    for i in xrange(0 , title_num1):   
                        url1=Times+title[i]['href']   #文章網址
                        f.write(url1  + '\n')  #寫入text

C:\Users\BIG DATA\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


# 取出連結 寫入json檔(一個月)     

## 3/25 

In [2]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import json
import ast  #轉換成json需要套件
import string

# 標題,類別,內文,關鍵詞,新聞連結網址,日期(格式yyyyMMdd)


def wr_all(url):  #方法 大部分的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')[0].text.encode('utf-8')
    date1 =  soup.select('#newstext span ')[0].text.encode('utf-8')
    category = soup.select('.guide  a ')[1].text.encode('utf-8')
    comp = " LibertyTimes"
    date = string.replace(date1, '-', '')[0:8]
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文

    return tital,date,category, page,comp
        
def wr_sport(url):  #方法 運動類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)    
    essay = soup.select('h4 , p')
    tital = soup.select('.Btitle ')[0].text.encode('utf-8')
    date1 =  soup.select('.news_content .date ')[0].text.encode('utf-8')
    category = "體育"
    comp = " LibertyTimes"
    date = string.replace(date1, '/', '')[0:8]

    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文  
        
    return tital,date,category, page,comp

def wr_ent(url):  #方法 娛樂類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('p')
    tital = soup.select('.news_content h1 ')[0].text.encode('utf-8')
    date1 =  soup.select('.news_content .date')[0].text.encode('utf-8')
    date = string.replace(date1, '/', '')[0:8]
    category = "娛樂"
    comp = " LibertyTimes"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文   
    return tital,date,category, page,comp

def wr_local(url): #方法 地方類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('#newstext p , h4')
    tital = soup.select('h1 ')[0].text.encode('utf-8')
    date1 =  soup.select('#newstext span ')[0].text.encode('utf-8')
    category = "地方"
    comp = " LibertyTimes"
    date = string.replace(date1, '-', '')[0:8]
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文
    return tital,date,category, page,comp
    
def wr_talk(url): # 方法 言論類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('.cont p')
    tital = soup.select('h2 ')[0].text.encode('utf-8')
    date1 =  soup.select('.writer span')[0].text.encode('utf-8')
    date = string.replace(date1, '-', '')[0:8]
    category = "言論"
    comp = " LibertyTimes"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文    
    return tital,date,category, page,comp

 
    


for i in range(3,4): # 1-3月
    all_json=""
    with open('C:/Users/BIG DATA/Desktop/python/essay/20160{}.txt'.format(i), 'r') as f:
        for line in f.readlines():
            url = line.strip()
            m = re.match('.*news/(\w+)/paper/(\w+)' ,url)  #正規表達法 切出m.group(1)類別 與 m.group(2)網頁最後的數字(用來當檔名)
            print m.group(1)
            if m.group(1) == 'sports': #判斷是否為體育類
                x=  wr_sport(url) #使用方法
                data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
                all_json+=str(data)+','
                time.sleep(1.5)
                
            elif m.group(1) == 'entertainment':#判斷是否為娛樂類
                x= wr_ent(url) #使用方法
                data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
                all_json+=str(data)+','
                time.sleep(1.5)


            elif m.group(1) =='local': #判斷是否為地方類
                x= wr_local(url) #使用方法
                data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
                all_json+=str(data)+','
                time.sleep(1.5)

            elif m.group(1) =='opinion': #判斷是否為言論類
                x= wr_talk(url) #使用方法
                data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
                all_json+=str(data)+','
                time.sleep(1.5)
 
            else:
                x= wr_all(url) #使用方法
                data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
                all_json+=str(data)+','
                
                time.sleep(1.5)
               



    a = eval(all_json)                       #AST套件 轉換 all_json 成 json
    d1 =  json.dumps(a)
    with open('C:/Users/BIG DATA/Desktop/python/essay/json/20160{}.json'.format(i), "w") as f:
        f.write(d1)
        f.close()
    data.clear()

focus
focus
focus
focus
focus
focus
focus
focus
focus
focus
focus
focus
politics
politics
politics
politics
politics
politics
politics
politics
politics
politics
politics
politics
politics
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
society
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
local
life
life
life
life
life
life
life
life
life
life
life
life
life
life
life
life
life
life
life
opinion
opinion
opinion
opinion
opinion
opinion
opinion
opinion
opinion
world
world
world
world
world
wo

In [ ]:
a=eval(all_json)                       #AST套件 轉換 all_json 成 json
d1 =  json.dumps(a)
d =  json.loads(d1)
for z in range(0,len(d)):
    print d[z]['tital'],d[z]['category'],d[z]['url'],d[z]['date']  #查詢KEY 

In [ ]:
import json
with open('.json', 'r') as f:
    d =  json.loads(d1)
    for i in range(0,len(d)):
        print d[i]['url'],d[i]['title'],d[i]['date'],d[i]['category'],d[i]['content'],d[i]['keyw']


# ----------------------------------------------------------

In [45]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import json
import ast  #轉換成json需要套件
import string

data={}

url = "http://sports.ltn.com.tw/news/breakingnews/1674348"
res = requests.get(url)
res.encoding = "UTF-8"
soup = bs(res.text)    
essay = soup.select('h4 , p')
tital = soup.select('.Btitle ')[0].text.encode('utf-8')
date1 =  soup.select('.news_content .date ')[0].text.encode('utf-8')
category = "體育"
comp = " LibertyTimes"
date = string.replace(date1, '/', '')[0:8]

page=""                          #給一個字串,存放內文用           
for div3 in essay[0:(len(essay))]:
    page += div3.text.encode('utf-8')       #加入內文  

data={'title':tital.decode('utf-8'),'date':date.decode('utf-8')}



In [46]:
data['date']


u'20160424'

In [2]:
import os
import string
x=[]
s='[' #建立多個字典
for dirPath, dirNames, fileNames in os.walk("E:/TEST/"):
    #print fileNames
    a = [dirPath, dirNames, fileNames]
    x.append(a)


In [3]:
import re

ff = open("newsports.txt",'w') 
a=0
for f in fileNames:
    path = x[0][0]
    road = os.path.join(path, f)
    with open(road,'r') as f:
        for line in f.readlines():
            url = line.strip()
            #print url
            m = re.match('.*news/(\w+)/paper/(\w+)' ,url)
            if m.group(1) == 'sports': 
                ff.write(url+ '\n')
                a=a+1


In [4]:
ff.close

<function close>

In [12]:
print a


27147


# ----------------------------------

In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import json
import ast  #轉換成json需要套件
import string

def wr_sport(url):  #方法 運動類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)    
    essay = soup.select('h4 , p')
    tital = soup.select('.Btitle ')[0].text.encode('utf-8')
    date1 =  soup.select('.news_content .date ')[0].text.encode('utf-8')
    category = "體育"
    comp = " LibertyTimes"
    date = string.replace(date1, '/', '')[0:8]

    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文  
        
    return tital,date,category, page,comp
# 標題,類別,內文,關鍵詞,新聞連結網址,日期(格式yyyyMMdd)


all_json=""
for i in xrange(1,5):
    with open('newsports{}.txt'.format(i), 'r') as f:
        for line in f.readlines():
            url = line.strip()
            x=  wr_sport(url) #使用方法
                data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
                all_json+=str(data)+','
                time.sleep(1.5)

    a = eval(all_json)                       #AST套件 轉換 all_json 成 json
    d1 =  json.dumps(a)
    with open('newsports{}.json'.format(i), "w") as f:
        f.write(d1)
        f.close()
    data.clear()            
        

In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import json
import ast  #轉換成json需要套件
import string

def wr_ent(url):  #方法 娛樂類的網頁版型
    res = requests.get(url)
    res.encoding = "UTF-8"
    soup = bs(res.text)
    essay = soup.select('p')
    tital = soup.select('.news_content h1 ')[0].text.encode('utf-8')
    date1 =  soup.select('.news_content .date')[0].text.encode('utf-8')
    date = string.replace(date1, '/', '')[0:8]
    category = "娛樂"
    comp = " LibertyTimes"
    page=""                          #給一個字串,存放內文用           
    for div3 in essay[0:(len(essay))]:
        page += div3.text.encode('utf-8')       #加入內文   
    return tital,date,category, page,comp

all_json=""
for i in xrange(1,5):
    with open('newentertainment{}.txt'.format(i), 'r') as f:
        for line in f.readlines():
            url = line.strip()
            x= wr_ent(url) #使用方法
            data ={'title' :x[0],'category' : x[2],'content' : x[3],'url':url,'date':x[1],'comp':x[4],'keyw':'','hitcount':0}
            all_json+=str(data)+','
            time.sleep(1.5)

    a = eval(all_json)                       #AST套件 轉換 all_json 成 json
    d1 =  json.dumps(a)
    with open('newentertainment{}.json'.format(i), "w") as f:
        f.write(d1)
        f.close()
    data.clear()            
                    